In [18]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from datasets import load_dataset
import matplotlib.pyplot as plt

In [19]:
# prepare dataset
class CatsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            temp_dataset = load_dataset("cifar10")['train']
        else:
            temp_dataset = load_dataset("cifar10")['test']
        
        self.dataset = []
        for i in range(len(temp_dataset)):
            if temp_dataset[i]['label'] == 3:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']).expand((3, -1, -1)) / 255.0
                self.dataset.append([img, torch.tensor([0])])

            elif torch.rand((1, )) < 1/9:
                img = torchvision.transforms.PILToTensor()(temp_dataset[i]['img']).expand((3, -1, -1)) / 255.0
                self.dataset.append([img, torch.tensor([1])])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = torchvision.transforms.RandomHorizontalFlip()(image)
        image = torchvision.transforms.RandomResizedCrop((32, 32), scale=(4/5, 5/4), ratio=(4/5, 5/4))(image)
        return (image, label)

In [20]:
train_dataset = CatsDataset(train=True)
len(train_dataset)

9947

In [21]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=256,
    shuffle=True
)

In [22]:
dev = torch.device("cpu")
if torch.cuda.is_available():
    dev = torch.device("cuda")
elif torch.backends.mps.is_available():
    dev = torch.device("mps")

print(f"Using device: {dev}")

Using device: mps


In [23]:
model = nn.Sequential(
    nn.Conv2d(3, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Conv2d(64, 64, 3),
    nn.MaxPool2d((3, 3)),
    nn.LeakyReLU(),
    nn.Flatten(),
    nn.Linear(256, 64),
    nn.LeakyReLU(),
    nn.Linear(64, 2)
).to(dev)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [24]:
for epoch in range(100):
    s = 0
    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(dev)
        labels = labels.to(dev)
        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        labels = labels.view(-1)
        # Compute the loss and its gradients
        loss = criterion(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        s += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {s/len(train_dataloader)}")

torch.save(model.to(dev).state_dict(), "model.pt")

Epoch 1 Loss: 0.6677583624155093
Epoch 2 Loss: 0.5944579686873999
Epoch 3 Loss: 0.556195385945149
Epoch 4 Loss: 0.5427723717995179
Epoch 5 Loss: 0.5270934937856137
Epoch 6 Loss: 0.5302112423456632
Epoch 7 Loss: 0.5126972939723577
Epoch 8 Loss: 0.5027764997421167
Epoch 9 Loss: 0.4952952953485342
Epoch 10 Loss: 0.4898380637168884
Epoch 11 Loss: 0.48987727898817796
Epoch 12 Loss: 0.48434199660252303
Epoch 13 Loss: 0.4750880492039216
Epoch 14 Loss: 0.47877767987740344
Epoch 15 Loss: 0.4683620188480768
Epoch 16 Loss: 0.4676677202567076
Epoch 17 Loss: 0.4642810271336482
Epoch 18 Loss: 0.46168935069671047
Epoch 19 Loss: 0.449918147845146
Epoch 20 Loss: 0.44952973417746717
Epoch 21 Loss: 0.4414935249548692
Epoch 22 Loss: 0.4363453533404913
Epoch 23 Loss: 0.4356663685578566
Epoch 24 Loss: 0.43145970961986446
Epoch 25 Loss: 0.42336919674506557
Epoch 26 Loss: 0.4198306256379837
Epoch 27 Loss: 0.42206848279023784
Epoch 28 Loss: 0.41971343984970677
Epoch 29 Loss: 0.41924567100329274
Epoch 30 Loss: 

In [25]:
test_dataset = CatsDataset(train=False)

In [26]:
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=64,
    shuffle=True
)

In [27]:
good = 0
a = 0
for i, data in enumerate(test_dataloader):
    # Every data instance is an input + label pair
    inputs, labels = data
    inputs = inputs.to(dev)
    labels = labels.to(dev)
    outputs = model(inputs)
    labels = labels.view(-1)
    
    for i in range(len(outputs)):
        good += torch.argmax(outputs[i]) == labels[i]
        a += 1

print(good / a)

tensor(0.7985, device='mps:0')
